In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
train=ImageDataGenerator(rescale=1/255)
validation=ImageDataGenerator(rescale=1/255)

In [3]:
train_dataset=train.flow_from_directory('G:/Deep Learning/dermnet_Dataset/train_dataset', 
        target_size=(150, 150), 
        batch_size=20,
        class_mode='categorical') 
train_dataset.class_indices

Found 568 images belonging to 4 classes.


{'acne': 0, 'eczema': 1, 'psoriasis': 2, 'rosacea': 3}

In [5]:
validation_dataset=validation.flow_from_directory(
      'G:/Deep Learning/dermnet_Dataset/validation_dataset',
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')

Found 157 images belonging to 4 classes.


In [6]:
X_train, y_train = next(train_dataset)
X_test, y_test = next(validation_dataset)

In [7]:

y_train.shape

(20, 4)

In [8]:
X_train=X_train.reshape(len(X_train),150,150,3)
X_test=X_test.reshape(len(X_test),150,150,3)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(150,150,3)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
      keras.layers.Conv2D(
        filters=hp.Int('conv_3_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_3_kernel', values = [3,5]),
        activation='relu'
      ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(
        units=hp.Int('dense_2_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(4, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
             loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [17]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=3,directory='disease5',project_name="Skin disease")

In [ ]:
tuner_search.search(X_train,y_train,epochs=100, validation_data=(X_test, y_test))

Train on 20 samples, validate on 20 samples
Epoch 1/100
20/20 [==============================] - 53s 3s/sample - loss: 1.3609 - accuracy: 0.4000 - val_loss: 6381.5557 - val_accuracy: 0.4000
Epoch 2/100
20/20 [==============================] - 7s 355ms/sample - loss: 4833.0552 - accuracy: 0.4000 - val_loss: 331.9403 - val_accuracy: 0.2500
Epoch 3/100
20/20 [==============================] - 5s 267ms/sample - loss: 246.1407 - accuracy: 0.3500 - val_loss: 16.3831 - val_accuracy: 0.4000
Epoch 4/100
20/20 [==============================] - 5s 262ms/sample - loss: 16.4717 - accuracy: 0.4000 - val_loss: 4.0790 - val_accuracy: 0.2500
Epoch 5/100
20/20 [==============================] - 5s 260ms/sample - loss: 2.8937 - accuracy: 0.3500 - val_loss: 67.8514 - val_accuracy: 0.2500
Epoch 6/100
20/20 [==============================] - 5s 255ms/sample - loss: 50.6981 - accuracy: 0.3500 - val_loss: 1.3567 - val_accuracy: 0.3000
Epoch 7/100
20/20 [==============================] - 5s 255ms/sample - los

Epoch 57/100
20/20 [==============================] - 5s 252ms/sample - loss: 1.0812 - accuracy: 0.4000 - val_loss: 1.4809 - val_accuracy: 0.4000
Epoch 58/100
20/20 [==============================] - 5s 263ms/sample - loss: 1.0810 - accuracy: 0.4000 - val_loss: 1.4852 - val_accuracy: 0.4000
Epoch 59/100
20/20 [==============================] - 5s 255ms/sample - loss: 1.0809 - accuracy: 0.4000 - val_loss: 1.4894 - val_accuracy: 0.4000
Epoch 60/100
20/20 [==============================] - 5s 256ms/sample - loss: 1.0809 - accuracy: 0.4000 - val_loss: 1.4931 - val_accuracy: 0.4000
Epoch 61/100
20/20 [==============================] - 5s 254ms/sample - loss: 1.0810 - accuracy: 0.4000 - val_loss: 1.4959 - val_accuracy: 0.4000
Epoch 62/100
20/20 [==============================] - 5s 257ms/sample - loss: 1.0811 - accuracy: 0.4000 - val_loss: 1.4974 - val_accuracy: 0.4000
Epoch 63/100
20/20 [==============================] - 5s 261ms/sample - loss: 1.0811 - accuracy: 0.4000 - val_loss: 1.4976 -

Train on 20 samples, validate on 20 samples
Epoch 1/100


In [16]:
model=tuner_search.get_best_models(num_models=1)[0]

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 144, 144, 48)      38448     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 142, 142, 64)      27712     
_________________________________________________________________
flatten (Flatten)            (None, 1290496)           0         
_________________________________________________________________
dense (Dense)                (None, 96)                123887712 
_________________________________________________________________
dense_1 (Dense)              (None, 112)               10864     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 3